In [78]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

import yfinance as yf

from scipy.optimize import Bounds
from scipy.optimize import LinearConstraint
from scipy.optimize import minimize

from Data_Retriever import DataRetriever as DatRet
from Markowitz_PT import MarkowitzPT as MPT
from Menchero_OGA import MencheroOGA as MOGA

In [51]:
# Define number of trades
trading_n = 30

# Define number of days used to calcualte MPT
history_usage = 100

# Defining pool variables
num_sectors = 5
num_stocks_peer_sector = 4

In [52]:
# Defining stock pool
ticker_df =  pd.DataFrame()
ticker_df["Petroleum"] = ["EQNR.OL", "AKRBP.OL", "SUBC.OL", "BWO.OL",]
ticker_df["Seafood (food)"] = ["ORK.OL", "MOWI.OL", "SALM.OL", "LSG.OL"]
# ticker_df["Materials"] = ["NHY.OL", "YAR.OL", "RECSI.OL", "BRG.OL"]
ticker_df["Technologies"] = ["TEL.OL", "NOD.OL", "ATEA.OL", "BOUV.OL"]
ticker_df["Financial"] = ["STB.OL", "DNB.OL", "GJF.OL", "AKER.OL"]
ticker_df["Shipping"] = ["WAWI.OL", "SNI.OL", "BELCO.OL", "ODF.OL"]
ticker_df

,Petroleum,Seafood (food),Technologies,Financial,Shipping
0,EQNR.OL,ORK.OL,TEL.OL,STB.OL,WAWI.OL
1,AKRBP.OL,MOWI.OL,NOD.OL,DNB.OL,SNI.OL
2,SUBC.OL,SALM.OL,ATEA.OL,GJF.OL,BELCO.OL
3,BWO.OL,LSG.OL,BOUV.OL,AKER.OL,ODF.OL


In [53]:
# Defining ESG scores for respective securities
esg_scores = [36.6, 35.3, 17.9, 18, 
              18, 21.2, 18.7, 29.2, 
            #   15.7, 25.6, 25.6, 18.4, 
              19.8, 13.8, 18.1, 19, 
              17.2, 14, 17.2, 19.5, 
              19.7, 21.2, 26.8, 19.3]

In [91]:
import pandas as pd
import numpy as np



class MencheroOGA():
    """
    A class for performing frequency analysis using the Optimized Geometric Attribution (OGA) model for 
    portfolio performance measurement, made by Menchero (2005).

    Attributes:
        benchmark_w (list): A nested list containing benchmark weights over time for each stock in each sector.
        experimental_w (list): A nested list containing experimental/portfolio weights over time for each stock in each sector.
        returns (list): A nested list of stock returns over time for each stock in each sector.
        n_optimizations (int): The number of time periods for which optimization is performed.
        n_sectors (int): The number of sectors in the portfolio.
        n_stocks (int): The number of stocks per sector.
        allocation_effects (list): A list storing the computed allocation effects over all time periods.
        selection_effects (list): A list storing the computed selection effects over all time periods.
    
    Methods:
        __init__(returns, benchmark_w, experimental_w):
            Initializes the MencheroOGA object with given returns, benchmark weights, and experimental weights.
        
        analyzer_at_time_t(ret, we, wb):
            Computes optimized allocation and selection effects at a given time step.
        
        frequency_analyser():
            Conducts frequency analysis over the entire optimization period and calculates the 
            allocation and selection effects for each time step.
    """

    def __init__(self, experimental_w, n_sectors, n_stocks_per_sector):
        """
        Initializes the MencheroOGA class with portfolio returns, benchmark weights, and experimental weights.

        Args:
            returns (list): A nested list of returns for each stock in each sector over multiple time periods.
            benchmark_w (list): A nested list of benchmark weights corresponding to stocks over multiple time periods.
            experimental_w (list): A nested list of experimental/portfolio weights corresponding to stocks over multiple time periods.
        """
        self.benchmark_w =  pd.read_csv("../Data/MPT_weights.csv")
        self.experimental_w = experimental_w
        self.returns = pd.read_csv("../Data/MPT_weights.csv")

        self.n_optimizations: int = self.benchmark_w.shape[0]
        
        self.n_sectors = n_sectors
        self.n_stocks = n_stocks_per_sector

        self.allocation_effects: list = None
        self.selection_effects: list = None


    def analyzer_at_time_t(self, ret:list,  we:list, wb:list):
        """
        Performs optimized geometric attribution analysis for a single time period.

        Args:
            ret (list): A list of returns for each stock at time t.
            we (list): A list of portfolio weights for each stock at time t.
            wb (list): A list of benchmark weights for each stock at time t.

        Returns:
            list: A list containing two elements:
                  - sel_opt (numpy.ndarray): Optimized selection effects for all sectors at time t.
                  - all_opt (numpy.ndarray): Optimized allocation effects for all sectors at time t.
        """
        # Returns and weights on sector level for  benchmark
        w_b = np.array([sum(wb[int(i*self.n_stocks):int((i+1)*self.n_stocks)]) for i in range(self.n_sectors)])
        r_b = np.array([wb[int(i*self.n_stocks):int((i+1)*self.n_stocks)] @ ret[int(i*self.n_stocks):int((i+1)*self.n_stocks)] for i in range(self.n_sectors)])/w_b

        # Returns and weights on sector level for portfolio
        w_e = np.array([sum(we[int(i*self.n_stocks):int((i+1)*self.n_stocks)]) for i in range(self.n_sectors)])
        r_e = np.array([we[int(i*self.n_stocks):int((i+1)*self.n_stocks)] @ ret[int(i*self.n_stocks):int((i+1)*self.n_stocks)] for i in range(self.n_sectors)])/w_e

        # Total portfolio and benchmark return
        Re = np.dot(w_e, r_e)
        Rb = np.dot(w_b, r_b)

        # Naked allocation and selection effects
        sel_nkd = ((1+w_e*r_e) / (1+ w_e*r_b)) -1
        all_nkd = ((1 + (w_e-w_b)*r_b) /  (1+(w_e-w_b)*Rb)) -1

        # Q value
        q_top =  np.log(1+Re) - np.log(1+Rb) - sum(np.log((1+sel_nkd)*(1+all_nkd)))
        q_bottom = sum(np.log(1+sel_nkd)**2) + sum(np.log(1+all_nkd)**2)
        q_tot =  q_top/q_bottom
        
        # Perturbation terms
        gam_sel  = q_tot * np.log(1+sel_nkd)**2
        gam_all = q_tot * np.log(1+all_nkd)**2

        # Optimized selection and allocation effects
        sel_opt = (1+sel_nkd) * np.e**(gam_sel) - 1
        all_opt = (1+all_nkd) * np.e**(gam_all) - 1

        return [sel_opt, all_opt]        



    def frequency_analyser(self):
        """
        Performs frequency analysis over multiple time periods to calculate selection and allocation effects.

        Returns:
            str: A success message indicating that the data was successfully generated.
        
        Side Effects:
            - Updates the 'allocation_effects' and 'selection_effects' attributes of the class.
        """

        # return_array =  [pd.Series(stock) for sector in self.returns for stock in sector]
        # return_df = pd.DataFrame(return_array)
        # return_tdf = return_df.T

        # relevant_return_list = [return_tdf.iloc[-+i] for i in range(self.n_optimizations)]
        relevant_return_list = [self.returns.iloc[-(self.n_optimizations+time)] for time in range(self.n_optimizations)]

        allocation_list = []
        selection_list = []
        for time in range(self.n_optimizations):
            effects = self.analyzer_at_time_t(relevant_return_list[time], 
                                              np.array(self.experimental_w.iloc[time]),
                                              np.array(self.benchmark_w[time][0]))

            selection_list.append(effects[0])
            allocation_list.append(effects[1])
        
        self.allocation_effects = np.concatenate(allocation_list)
        self.selection_effects = np.concatenate(selection_list)
        print("--Frequency analysis performed succesfully--")


In [ ]:
bravo = MencheroOGA(b)

In [82]:
df = pd.read_csv("../Data/StockReturns.csv")

In [89]:
df.iloc[-trading_n:].shape[0]

30